<a href="https://colab.research.google.com/github/Homura-san/pyspark/blob/main/TrabTopBD_op2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
# Fazendo download
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

# Descompactando os arquivos
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [3]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [4]:
import os

# Definindo a variável de ambiente do Java
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Definindo a variável de ambiente do Spark
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [5]:
# instalando a findspark
!pip install -q findspark

In [6]:
# Importando a findspark
import findspark

# Iniciando o findspark
findspark.init()

In [7]:
# importando o pacote necessário para iniciar uma seção Spark
from pyspark.sql import SparkSession

# iniciando o spark context
sc = SparkSession.builder.master('local[*]').getOrCreate()

# Verificando se a sessão foi criada
sc

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Preparando datasets

In [50]:
import pandas as pd
from pyspark.sql import functions as F
from pyspark.ml.feature import PCA
#from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline

In [9]:
dadosCovid = sc.read.csv(
    path = "/content/drive/My Drive/dadosGerais2021.csv", 
    inferSchema = True, 
    header = True,
    sep = ',', 
    encoding = "UTF-8")

In [10]:
vacinacaoCovid = sc.read.csv(
    path = "/content/drive/My Drive/vacinacaoPorDia.csv", 
    inferSchema = True, 
    header = True,
    sep = ',', 
    encoding = "UTF-8")

In [ ]:
type(dadosCovid)

pyspark.sql.dataframe.DataFrame

In [ ]:
dadosCovid.show()

+-------------------+------------------+--------------------+-----------------------+--------+----+-----------------------+-----+-----+---------------+--------+--------------------+---------------------------------------------+-------------------------------------------+----------------------------------------------------------------------+--------+---------------+--------------------------+----------+
|DATA_DA_NOTIFICACAO|DATA_DE_NASCIMENTO| CLASSIFICACAO_FINAL|MUNICIPIO_DE_RESIDENCIA|RACA_COR|SEXO|E_PROFISSIONAL_DE_SAUDE|FEBRE|TOSSE|DOR_DE_GARGANTA|DISPNEIA|              OUTROS|DOENCAS_RESPIRATORIAS_CRONICAS_DESCOMPENSADAS|DOENCAS_RENAIS_CRONICAS_EM_ESTAGIO_AVANCADO|PORTADOR_DE_DOENCAS_CROMOSSOMICAS_OU_ESTADO_DE_FRAGILIDADE_IMUNOLOGICA|DIABETES|IMUNOSSUPRESSAO|DOENCAS_CARDIACAS_CRONICAS|    STATUS|
+-------------------+------------------+--------------------+-----------------------+--------+----+-----------------------+-----+-----+---------------+--------+--------------------+-------

In [83]:
dadosCovid.groupby('CLASSIFICACAO_FINAL').count().show()

+--------------------+------+
| CLASSIFICACAO_FINAL| count|
+--------------------+------+
|Confirmado por Cr...|   248|
|          Confirmado|    12|
|                null| 14297|
|Vinculo- Epidemio...|   113|
|Confirmado Clínic...|   546|
|        Laboratorial|  7789|
|Confirmado Labora...|100935|
|Síndrome Gripal N...|     8|
|             Clínico|   184|
|                   4|     6|
|Confirmado Clínic...|    13|
|          Descartado|   766|
+--------------------+------+



In [54]:
ab = dadosCovid.filter(F.col('CLASSIFICACAO_FINAL').contains('Vinculo-'))
ab.select(['CLASSIFICACAO_FINAL']).show()

+--------------------+
| CLASSIFICACAO_FINAL|
+--------------------+
|Vinculo- Epidemio...|
|Vinculo- Epidemio...|
|Vinculo- Epidemio...|
|Vinculo- Epidemio...|
|Vinculo- Epidemio...|
|Vinculo- Epidemio...|
|Vinculo- Epidemio...|
|Vinculo- Epidemio...|
|Vinculo- Epidemio...|
|Vinculo- Epidemio...|
|Vinculo- Epidemio...|
|Vinculo- Epidemio...|
|Vinculo- Epidemio...|
|Vinculo- Epidemio...|
|Vinculo- Epidemio...|
|Vinculo- Epidemio...|
|Vinculo- Epidemio...|
|Vinculo- Epidemio...|
|Vinculo- Epidemio...|
|Vinculo- Epidemio...|
+--------------------+
only showing top 20 rows



In [ ]:
#dadosCovid.groupby('DATA_DA_NOTIFICACAO').count().show(40)

In [70]:
casosNaoNulos = dadosCovid.filter(dadosCovid['CLASSIFICACAO_FINAL'] != 'null')
casosNaoDesc = casosNaoNulos.filter(dadosCovid.CLASSIFICACAO_FINAL != 'Descartado')

In [71]:
casosNao4 = casosNaoDesc.filter(dadosCovid.CLASSIFICACAO_FINAL != '4')

In [98]:
casosEpid = casosNao4.filter(dadosCovid.CLASSIFICACAO_FINAL.contains('Vinculo-') == False)
casosConfirmados = casosEpid.count()
# Imprimindo total de casos confirmados
print(casosConfirmados)

109735


In [ ]:
vacinacaoCovid.show()

+---------+-------+
|    DATAS|NUMEROS|
+---------+-------+
|19-JAN-21|   1396|
|20-JAN-21|   3918|
|21-JAN-21|   3404|
|22-JAN-21|   2687|
|23-JAN-21|   2875|
|24-JAN-21|   3242|
|25-JAN-21|   3198|
|26-JAN-21|   2244|
|27-JAN-21|   1643|
|28-JAN-21|   2874|
|29-JAN-21|   8649|
|30-JAN-21|  10202|
|31-JAN-21|   7144|
|01-FEB-21|   4154|
|02-FEB-21|   2869|
|03-FEB-21|   2782|
|04-FEB-21|   5058|
|05-FEB-21|   7966|
|06-FEB-21|   8786|
|07-FEB-21|   1357|
+---------+-------+
only showing top 20 rows



In [100]:
janeiro = vacinacaoCovid.filter(vacinacaoCovid.DATAS.contains('JAN-21'))
janeiro.show()
# Como eu posso somar os valores de cada linha??

+---------+-------+
|    DATAS|NUMEROS|
+---------+-------+
|19-JAN-21|   1396|
|20-JAN-21|   3918|
|21-JAN-21|   3404|
|22-JAN-21|   2687|
|23-JAN-21|   2875|
|24-JAN-21|   3242|
|25-JAN-21|   3198|
|26-JAN-21|   2244|
|27-JAN-21|   1643|
|28-JAN-21|   2874|
|29-JAN-21|   8649|
|30-JAN-21|  10202|
|31-JAN-21|   7144|
+---------+-------+



In [ ]:
dadosCovid.printSchema()

root
 |-- DATA_DA_NOTIFICACAO: string (nullable = true)
 |-- DATA_DE_NASCIMENTO: string (nullable = true)
 |-- CLASSIFICACAO_FINAL: string (nullable = true)
 |-- MUNICIPIO_DE_RESIDENCIA: string (nullable = true)
 |-- RACA_COR: string (nullable = true)
 |-- SEXO: string (nullable = true)
 |-- E_PROFISSIONAL_DE_SAUDE: string (nullable = true)
 |-- FEBRE: string (nullable = true)
 |-- TOSSE: string (nullable = true)
 |-- DOR_DE_GARGANTA: string (nullable = true)
 |-- DISPNEIA: string (nullable = true)
 |-- OUTROS: string (nullable = true)
 |-- DOENCAS_RESPIRATORIAS_CRONICAS_DESCOMPENSADAS: string (nullable = true)
 |-- DOENCAS_RENAIS_CRONICAS_EM_ESTAGIO_AVANCADO: string (nullable = true)
 |-- PORTADOR_DE_DOENCAS_CROMOSSOMICAS_OU_ESTADO_DE_FRAGILIDADE_IMUNOLOGICA: string (nullable = true)
 |-- DIABETES: string (nullable = true)
 |-- IMUNOSSUPRESSAO: string (nullable = true)
 |-- DOENCAS_CARDIACAS_CRONICAS: string (nullable = true)
 |-- STATUS: string (nullable = true)



In [ ]:
from pyspark.sql.types import (
    StringType, BooleanType, IntegerType, FloatType, DateType
)

coltype_map = {
    "DATA_DA_NOTIFICACAO": DateType(),
    
}


In [ ]:
course_df2 = dadosCovid.withColumn("DATA_DA_NOTIFICACAO", 
                                  dadosCovid["DATA_DA_NOTIFICACAO"]
                                  .cast(DateType()))
course_df2.printSchema()

root
 |-- DATA_DA_NOTIFICACAO: date (nullable = true)
 |-- DATA_DE_NASCIMENTO: string (nullable = true)
 |-- CLASSIFICACAO_FINAL: string (nullable = true)
 |-- MUNICIPIO_DE_RESIDENCIA: string (nullable = true)
 |-- RACA_COR: string (nullable = true)
 |-- SEXO: string (nullable = true)
 |-- E_PROFISSIONAL_DE_SAUDE: string (nullable = true)
 |-- FEBRE: string (nullable = true)
 |-- TOSSE: string (nullable = true)
 |-- DOR_DE_GARGANTA: string (nullable = true)
 |-- DISPNEIA: string (nullable = true)
 |-- OUTROS: string (nullable = true)
 |-- DOENCAS_RESPIRATORIAS_CRONICAS_DESCOMPENSADAS: string (nullable = true)
 |-- DOENCAS_RENAIS_CRONICAS_EM_ESTAGIO_AVANCADO: string (nullable = true)
 |-- PORTADOR_DE_DOENCAS_CROMOSSOMICAS_OU_ESTADO_DE_FRAGILIDADE_IMUNOLOGICA: string (nullable = true)
 |-- DIABETES: string (nullable = true)
 |-- IMUNOSSUPRESSAO: string (nullable = true)
 |-- DOENCAS_CARDIACAS_CRONICAS: string (nullable = true)
 |-- STATUS: string (nullable = true)

